In [11]:
import os

def split_logs(file_path, output_dir):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


    current_file = None
    current_log = []

    with open(file_path, 'r') as log_file:
        for line in log_file:
            line = line.strip()

            if line.startswith('File:'):
                if current_file:
                    output_path = os.path.join(output_dir, f"log_{current_file}.log")
                    with open(output_path, 'w') as f:
                        f.write('\n'.join(current_log))


                current_file = line.split(': ')[1].strip()
                current_log = [line] 
            else:
                current_log.append(line)

        if current_file:
            output_path = os.path.join(output_dir, f"log_{current_file}.log")
            with open(output_path, 'w') as f:
                f.write('\n'.join(current_log))


file_path = 'output_se.log' 
output_dir = 'split_logs/final' 
split_logs(file_path, output_dir)


In [12]:
import csv

def process_log_file(file_path, output_csv):
    
    buffer_data = []
    current_buffer = None
    current_sums = {
        "Buffer": 0,
        "dwt2d-1": 0,
        "dwt2d-2": 0,
        "sha256+xor1": 0,
        "sha512+xor2": 0,
        "aes_encrypt": 0,
        "_se_encrypt": 0,
        "encrypt": 0
    }

    # current_sums = {
    #     "Buffer": 0,
    #     "_se_encrypt": 0,
    #     "encrypt": 0
    # }
    
    
    dwt2d_count = 0
    xor1_sum = 0
    xor2_sum = 0
    processing_xor1 = True

    with open(file_path, 'r') as log_file:
        for line in log_file:
            line = line.strip()


            if line.startswith("Buffer:"):
                
                if current_buffer is not None:
                    current_sums["Buffer"] = current_buffer
                    buffer_data.append(current_sums)


                current_buffer = int(line.split(": ")[1])
                current_sums = {
                    "Buffer": current_buffer,
                    "dwt2d-1": 0,
                    "dwt2d-2": 0,
                    "sha256+xor1": 0,
                    "sha512+xor2": 0,
                    "aes_encrypt": 0,
                    "_se_encrypt": 0,
                    "encrypt": 0
                }
                # current_sums = {
                #         "Buffer": 0,
                #         "_se_encrypt": 0,
                #         "encrypt": 0
                #     }
                dwt2d_count = 0
                xor1_sum = 0
                xor2_sum = 0
                processing_xor1 = True

            elif line.startswith("itr_count:"):
                pass

            elif line.startswith("dwt2d:"):
                dwt2d_count += 1
                if dwt2d_count == 1:
                    current_sums["dwt2d-1"] += float(line.split(": ")[1])
                elif dwt2d_count == 2:
                    current_sums["dwt2d-2"] += float(line.split(": ")[1])
                    dwt2d_count = 0

            elif line.startswith("sha256_with_key:"):
                sha256_value = float(line.split(": ")[1])
                xor1_sum += sha256_value
                processing_xor1 = True

            elif line.startswith("sha512_with_key:"):
                sha512_value = float(line.split(": ")[1])
                xor2_sum += sha512_value

            elif line.startswith("xor_with_sha_key:"):
                if processing_xor1:
                    xor1_sum += float(line.split(": ")[1])
                    current_sums["sha256+xor1"] += xor1_sum
                    xor1_sum = 0
                    processing_xor1 = False 
                else:
                    xor2_sum += float(line.split(": ")[1])
                    current_sums["sha512+xor2"] += xor2_sum
                    xor2_sum = 0

            elif line.startswith("aes_encrypt:"):
                current_sums["aes_encrypt"] += float(line.split(": ")[1])

            elif line.startswith("_se_encrypt:"):
                current_sums["_se_encrypt"] += float(line.split(": ")[1])
            
            elif line.startswith("encrypt:"):
                current_sums["encrypt"] += float(line.split(": ")[1])


        if current_buffer is not None:
            current_sums["Buffer"] = current_buffer
            buffer_data.append(current_sums)


    with open(output_csv, 'w', newline='') as csv_file:
        fieldnames = ["Buffer", "dwt2d-1", "dwt2d-2", "sha256+xor1", "sha512+xor2", "aes_encrypt", "_se_encrypt", "encrypt"]
        # fieldnames = ["Buffer", "_se_encrypt", "encrypt"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(buffer_data)

def process_all_files_in_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    
    for filename in os.listdir(input_dir):
        if filename.endswith(".log"):
            file_path = os.path.join(input_dir, filename)
            output_csv = os.path.join(output_dir, filename.replace('.log', '.csv'))
            process_log_file(file_path, output_csv)
            print(f"Processed {filename} into {output_csv}")


input_dir = 'split_logs/final'
output_dir = 'csv/final'
process_all_files_in_directory(input_dir, output_dir)


Processed log_256MB_image.png.log into csv/final/log_256MB_image.png.csv
Processed log_4MB_image.png.log into csv/final/log_4MB_image.png.csv
Processed log_2MB_image.png.log into csv/final/log_2MB_image.png.csv
Processed log_16MB_image.png.log into csv/final/log_16MB_image.png.csv
Processed log_8MB_image.png.log into csv/final/log_8MB_image.png.csv
Processed log_32MB_image.png.log into csv/final/log_32MB_image.png.csv
Processed log_64MB_image.png.log into csv/final/log_64MB_image.png.csv
Processed log_128MB_image.png.log into csv/final/log_128MB_image.png.csv
